In [1]:
import stormpy
import inspect
import interval_parser
import parser
import importlib

In [99]:
upomdp, upomdp_data, params = parser.main()

FullArgSpec(args=['symbolic_description', 'properties'], varargs=None, varkw=None, defaults=(None,), kwonlyargs=[], kwonlydefaults=None, annotations={})


In [101]:
from operator import itemgetter
from itertools import groupby

#states
states_list = ["s%s"%state for state in upomdp.states]

#actions
act_list = sorted(list(set([upomdp_data[i][1] for i in range(len(upomdp_data))])))

#states action list
states_act_list = [[ud[0],ud[1]] for ud in upomdp_data]
states_act_list = sorted(states_act_list, key=itemgetter(0))
states_act_list = [[k,list(set([a for (s,a) in list(g)]))] for k, g in (groupby(states_act_list, key=itemgetter(0)))]

#observations
obs_list = ["o%s"%(obs) for obs in list(set(upomdp.observations))]
obs_list.append("o_crash")
obs_list.append("o_deadlock")
obs_list = sorted(obs_list)
# I had to add observations for crash and deadlock actions (both now go to observation 0)

#observation states list
func1 = lambda s, o: len(obs_list) if ("crash_action" in states_act_list[s][1]) else (len(obs_list)+1) if ("deadlock_action" in states_act_list[s][1]) else o
func2 = lambda o: "o_crash" if (o == len(obs_list)) else "o_deadlock" if (o == (len(obs_list)+1)) else "o%s"%o
obs_states_list = list(zip(states_list, upomdp.observations))
obs_states_list = [(s,func1(int(s[1:]),o)) for (s,o) in obs_states_list]
obs_states_list = sorted(obs_states_list, key=itemgetter(1))
obs_states_list = [[func2(k),[s for (s,o) in list(g)]] for k, g in (groupby(obs_states_list, key=itemgetter(1)))]
obs_states_list = sorted(obs_states_list, key=itemgetter(0))

#observation action list
obs_act_list = [[o,s[0]] for [o,s] in obs_states_list]
obs_act_list = [[o,sorted(list(set(map(lambda x: x[1],list(filter(lambda x: "s%s"%x[0] == s, upomdp_data))))))] for [o,s] in obs_act_list]
obs_act_list = sorted(obs_act_list, key=itemgetter(0))

#intial belief
floor = lambda num, precision: ((num*10**precision)//1)/(10**precision)
ceil = lambda num, precision: -((-num*10**precision)//1)/(10**precision)
initial_belief = [["s%s"%s,[floor(1/len(upomdp.initial_states),3),ceil(1/len(upomdp.initial_states),3)]] for s in upomdp.initial_states]

#rewards
rewards_model_name = list(upomdp.reward_models.keys())[0]
assert upomdp.reward_models[rewards_model_name].has_state_action_rewards
rewards = upomdp.reward_models[rewards_model_name].state_action_rewards

rewards_list = [[s,a] for [s,a_list] in states_act_list for a in a_list]
rewards_list = [rewards_list[i]+[int(str(rewards[i]))] for i in range(len(rewards_list))]

with open("aircraft.txt", "w", newline="") as outfile:
    outfile.write("# States\n")
    outfile.write(" ".join(states_list)+"\n")
    outfile.write("\n# Observations\n")
    outfile.write(" ".join(obs_list)+"\n")
    outfile.write("\n# Actions\n")
    outfile.write(" ".join(act_list)+"\n")
    outfile.write("\n# Observations states list\n")
    for item in obs_states_list:
        outfile.write(" ".join(map(str,item[:-1]))+" "+" ".join(map(str,item[-1]))+"\n")
    outfile.write("\n# Observations actions list\n")
    for item in obs_act_list:
        outfile.write(" ".join(map(str,item[:-1]))+" "+" ".join(map(str,item[-1]))+"\n")
    outfile.write(f"\n# Initial belief\n{len(upomdp.initial_states)}\n")
    for item in initial_belief:
        outfile.write(" ".join(map(str,item[:-1]))+" "+" ".join(map(str,item[-1]))+"\n")
    outfile.write(f"\n# Reward\n{len(rewards_list)}\n")
    for item in rewards_list:
        outfile.write(" ".join(map(str,item))+"\n")
    outfile.write(f"\n# Transitions\n{len(upomdp_data)}\n")
    for item in upomdp_data:
        outfile.write(" ".join(map(str,item[:-1]))+" "+" ".join(map(str,item[-1]))+"\n")